Lectura y escritura de ficheros
===============================

### Sistemas de ficheros soportados
-   Igual que Hadoop, Spark soporta diferentes filesystems: Local, HDFS,
    Amazon S3

    -   En general, soporta cualquier fuente de datos que ofrezca un
        Hadoop InputFormat

-   También, acceso a bases de datos estructuradas o no estructuradas

    -   MySQL, Postgres, etc. mediante JDBC

    -   Apache Hive, HBase, Cassandra o Elasticsearch

### Formatos de fichero soportados

-   Spark puede acceder a diferentes tipos de ficheros:

    -   Texto plano, JSON, CSV, ficheros sequence, *protocol buffers* y
        *object files*

    -   Igual que Hadoop, soporta ficheros comprimidos (mismas
        consideraciones sobre formatos “splittables”)

### Ficheros de texto

In [1]:
# Lee todos los ficheros del directorio
# y crea una lista particionada de líneas
lineas = sc.textFile("datos/libros-mini/*")
words = lineas.flatMap(lambda x: x.split(" "))
# Salva el RDD words en varios ficheros de salida
# (un fichero por partición)
words.saveAsTextFile("file:///tmp/txtoutdir4")
        
# Lee ficheros y devuelve un par clave/valor
# clave->nombre fichero, valor->fichero completo
rdd=sc.wholeTextFiles("datos/libros-mini/*")
# Obtiene un lista clave/valor
# clave->nombre fichero, valor->numero de palabras
lista = rdd.mapValues(lambda x: len(x.split())).collect()
for libro in lista:
    print("El fichero {0:14s} tiene {1:6d} palabras"
          .format(libro[0].split("/")[-1], libro[1]))

El fichero pg9980.txt.bz2 tiene  34014 palabras
El fichero pg16625.txt.bz2 tiene 170900 palabras
El fichero pg5201.txt.bz2 tiene  49441 palabras
El fichero pg25807.txt.bz2 tiene  15014 palabras
El fichero pg32315.txt.bz2 tiene  46142 palabras
El fichero pg24536.txt.bz2 tiene 134016 palabras
El fichero pg2000.txt.bz2 tiene 384258 palabras
El fichero pg1619.txt.bz2 tiene 109878 palabras
El fichero pg7109.txt.bz2 tiene  35037 palabras
El fichero pg18005.txt.bz2 tiene  86446 palabras
El fichero pg17073.txt.bz2 tiene 309473 palabras
El fichero pg14329.txt.bz2 tiene 183777 palabras
El fichero pg25640.txt.bz2 tiene 207338 palabras
El fichero pg8870.txt.bz2 tiene  54348 palabras
El fichero pg17013.txt.bz2 tiene 396086 palabras


### Ficheros JSON

In [2]:
input = sc.textFile("datos/info.json")
import json
data = input.map(lambda x: json.loads(x))
(data.filter(lambda x: "client" in x and x["client"])
            .map(lambda x: json.dumps(x))
            .saveAsTextFile("file:///tmp/jsonoutdir"))

### Ficheros Sequence
Ficheros clave/valor usados en Hadoop

-   Sus elementos implementan la interfaz `Writable`

In [3]:
data = sc.parallelize([("a",2), ("b",5), ("a",8)])
# Salvamos el RDD como fichero secuence
data.saveAsSequenceFile("file:///tmp/sequenceoutdir")
# Lo leemos en otro RDD
rdd = sc.sequenceFile("file:///tmp/sequenceoutdir", 
                      "org.apache.hadoop.io.Text", 
                      "org.apache.hadoop.io.IntWritable")
print(rdd.collect())

[(u'a', 2), (u'a', 8), (u'b', 5)]


### Formatos de entrada/salida de Hadoop
Spark puede interactuar con cualquier formato de fichero soportado por Hadoop 
- Soporta las APIs “vieja” y “nueva”
- Permite acceder a otros tipos de almacenamiento (no fichero), p.e. HBase o MongoDB, a través de `saveAsHadoopDataSet` y/o `saveAsNewAPIHadoopDataSet`

In [4]:
# Salvamos el RDD como fichero de texto Hadoop (TextOutputFormat)
data.saveAsNewAPIHadoopFile("file:///tmp/hadoopfileoutdir", 
                            "org.apache.hadoop.mapreduce.lib.output.TextOutputFormat",
                            "org.apache.hadoop.io.Text",
                            "org.apache.hadoop.io.IntWritable")
# Lo leemos como fichero clave-valor Hadoop (KeyValueTextInputFormat)
rdd = sc.newAPIHadoopFile("file:///tmp/hadoopfileoutdir", 
                          "org.apache.hadoop.mapreduce.lib.input.KeyValueTextInputFormat",
                          "org.apache.hadoop.io.Text",
                          "org.apache.hadoop.io.IntWritable")
print(rdd.collect())

[(u'b', u'5'), (u'a', u'2'), (u'a', u'8')]


### Object files

Ficheros binarios que guardan cualquier tipo de RDDs (no solo
clave/valor)

-   Usan serialización Java (en Java y Scala, métodos
    `saveAsObjectFile()` y `objectFile()`)

-   En Python, se usa *pickle* (métodos `saveAsPickleFile()` y
    `pickleFile()`)

-   Solo útiles para comunicar trabajos Spark entre sí.

## Práctica 4

A partir del fichero apat63_99.txt, crear un fichero secuencia apat63_99.seq con clave=país (campo 4) valor=n_patente,año (campos 0 y 1), ambos de tipo string.

In [23]:
from test_helper import Test

#Borramos el directorio de salida si existe (comando Linux)
!rm -rf /tmp/apat63_99.seq

data = sc.textFile("datos/patentes-mini/apat63_99.txt")

prdd = data.map(lambda x: (x.split(",")[4].encode('utf-8'),x.split(",")[0].encode('utf-8')+\
                           ","+x.split(",")[1].encode('utf-8')))

pairs = prdd.filter(lambda x: x[0] != '\"COUNTRY\"')

#print (pairs.collect())

pairs.saveAsSequenceFile("file:///tmp/apat63_99.seq")

# Lo leemos para comprobar que se guardó bien en otro RDD
rdd = sc.sequenceFile("file:///tmp/apat63_99.seq", 
                      "org.apache.hadoop.io.Text", 
                      "org.apache.hadoop.io.Text")

#print(rdd.collect())

#Comprobamos que lo guardado en el sequence y lo leído son iguales
Test.assertEquals(pairs.collect(), rdd.collect())

#NOTA: Python detecta como iguales las dos colecciones porque todos los elementos son iguales
#aunque el primer RDD se codifique en UTF-8 y el segundo RDD se lea como Unicode. Para Python
#esto es indiferente y cuando se recorren los elementos se pueden decodificar o codificar a 
#UTF-8 o a cualquier otro formato si se desea.

1 test passed.
